# RAG using local models



https://python.langchain.com/docs/use_cases/question_answering/local_retrieval_qa

In [67]:
%pip install transformer langchainhub fake_useragent elasticsearch eland[pytorch]==8.10.1


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.4.14 requires tokenizers>=0.13.2, but you have tokenizers 0.12.1 which is incompatible.


  Using cached eland-8.10.1-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl (14.6 MB)
  Using cached torch-1.13.1-cp310-cp310-win_amd64.whl (162.6 MB)
  Using cached transformers-4.33.2-py3-none-any.whl.metadata (119 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.16.0-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
  Using cached torchvision-0.15.2-cp310-cp310-win_amd64.whl (1.2 MB)
  Using cached torchvision-0.15.1-cp310-cp310-win_amd64.whl (1.2 MB)
  Using cached torchvision-0.14.1-cp310-cp310-win_amd64.whl (1.1 MB)
Using cached eland-8.10.1-py3-none-any.whl (156 kB)
Using cached transformers-4.33.2-py3-none-any.whl (7.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: numpy
    Found exis

In [68]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
loader.requests_kwargs = {'verify':False}

data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

c:\ProgramData\miniconda3\envs\chatbot\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.score'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [69]:
from getpass import getpass
from langchain.vectorstores import ElasticsearchStore


In [70]:
# ES_URL = input('Elasticsearch URL: ')
ES_URL = input('Elasticsearch URL(ex:https://127.0.0.1:9200)')
ES_USER = "elastic" 
ES_USER_PASSWORD = getpass('elastic user PW: ')
CERT_PATH = input('Elasticsearch pem 파일 경로: ')
# pem 생성 방법: https://cdax.ch/2022/02/20/elasticsearch-python-workshop-1-the-basics/

In [71]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_USER, ES_USER_PASSWORD),
    ca_certs=CERT_PATH
)

In [79]:
# !eland_import_hub_model --url "https://127.0.0.1:9200" -u elastic -p elastic --ca-certs "D:\es\8.10.2\kibana-8.10.2\data\ca_1695716998259.crt" --hub-model-id "d:/Projects/es-lab-kr/s-core/models/distilbert-base-multilingual-cased" --task-type text_embedding --start --clear-previous

2023-10-20 20:43:49,416 INFO : Establishing connection to Elasticsearch
2023-10-20 20:43:49,648 INFO : Connected to cluster named 'elasticsearch' (version: 8.10.2)
2023-10-20 20:43:49,657 INFO : Loading HuggingFace transformer tokenizer and model 'd:/Projects/es-lab-kr/s-core/models/distilbert-base-multilingual-cased'
2023-10-20 20:43:50,491 WARNING : No sentence-transformers model found with name d:/Projects/es-lab-kr/s-core/models/distilbert-base-multilingual-cased. Creating a new one with MEAN pooling.
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
c:\ProgramData\miniconda3\envs\chatbot\lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. 

In [87]:
import os
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.common import es_version
from eland.ml.pytorch.transformers import TransformerModel

# 현재 경로 얻기
cwd = os.getcwd()
local_model_path = cwd + '/models/distilbert-base-multilingual-cased'


print(local_model_path)

# 모델 이름 및 작업 유형 설정
#tm = TransformerModel(local_model_path, "text_embedding")
es_cluster_version = es_version(client)
tm = TransformerModel(model_id=local_model_path, task_type="text_embedding", es_version=es_cluster_version)

# Elasticsearch용 modelID를 설정
es_model_id = "sentence-transformers_distilbert-base-multilingual-cased"

tmp_path = "models/sentence-transformers_distilbert-base-multilingual-cased"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

ptm = PyTorchModel(client, es_model_id)
ptm.stop()
ptm.delete()

ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config) 

ptm.start()

d:\Projects\es-lab-kr\s-core/models/distilbert-base-multilingual-cased
2023-10-20 21:08:46 - Load pretrained SentenceTransformer: d:\Projects\es-lab-kr\s-core/models/distilbert-base-multilingual-cased
2023-10-20 21:08:46 - No sentence-transformers model found with name d:\Projects\es-lab-kr\s-core/models/distilbert-base-multilingual-cased. Creating a new one with MEAN pooling.
2023-10-20 21:08:47 - Use pytorch device: cpu


Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
c:\ProgramData\miniconda3\envs\chatbot\lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


2023-10-20 21:08:51 - Resurrected node <Urllib3HttpNode(https://127.0.0.1:9200)> (force=False)
2023-10-20 21:08:54 - POST https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased/deployment/_stop [status:200 duration:3.494s]
2023-10-20 21:08:54 - Node <Urllib3HttpNode(https://127.0.0.1:9200)> has been marked alive after a successful request
2023-10-20 21:08:55 - DELETE https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased [status:200 duration:1.046s]
2023-10-20 21:08:56 - PUT https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased [status:200 duration:0.948s]


  0%|          | 0/515 [00:00<?, ? parts/s]

2023-10-20 21:08:57 - PUT https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased/definition/0 [status:200 duration:0.291s]
2023-10-20 21:08:57 - PUT https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased/definition/1 [status:200 duration:0.307s]
2023-10-20 21:08:58 - PUT https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased/definition/2 [status:200 duration:0.516s]
2023-10-20 21:08:58 - PUT https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased/definition/3 [status:200 duration:0.201s]
2023-10-20 21:08:58 - PUT https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased/definition/4 [status:200 duration:0.332s]
2023-10-20 21:08:59 - PUT https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased/definition/5 [status:200 duration:0.559s

In [88]:
from langchain.vectorstores import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

embeddings = ElasticsearchEmbeddings.from_es_connection(
    es_connection=client,
    model_id = es_model_id
)

vectorstore = ElasticsearchStore.from_documents(es_connection=client, documents=all_splits, embedding=embeddings, index_name="test-basic")

2023-10-20 21:13:22 - POST https://127.0.0.1:9200/_ml/trained_models/sentence-transformers_distilbert-base-multilingual-cased/_infer [status:409 duration:0.005s]
2023-10-20 21:13:22 - Node <Urllib3HttpNode(https://127.0.0.1:9200)> has failed for 1 times in a row, putting on 1 second timeout


ConflictError: ConflictError(409, 'status_exception', 'Trained model deployment [sentence-transformers_distilbert-base-multilingual-cased] is not allocated to any nodes')